### this notebok is deprecated, see evaluateModels.py instead.

In [1]:
import numpy as np
import tqdm
import torch
import argparse
from RAFT.flowNetsRAFT256_SS import RAFT256_SS
from RAFT.flowNetsRAFT256 import RAFT256
import matplotlib.pyplot as plt

from torchvision.utils import flow_to_image

import glob

In [2]:
device = torch.device('cuda', 0)
# _ = torch.set_grad_enabled(False)

In [3]:
parser = argparse.ArgumentParser()
parser.add_argument('-n', '--nodes', default=1, type=int,
                    help='number of compute nodes')
parser.add_argument('-g', '--gpus', default=2, type=int,
                    help='number of gpus per node')
parser.add_argument('--name', type=str, default='RAFT-PIV256_test_test1',
                    help='name of experiment')
parser.add_argument('--input_path_ckpt', type=str,
                    default='./precomputed_ckpts/RAFT256-PIV_ProbClass2/ckpt.tar',
                    help='path of already trained checkpoint')
parser.add_argument('--recover', type=eval, default=True,
                    help='Wether to load an existing checkpoint')
parser.add_argument('--output_dir_results', type=str, default='./test_results/',
                    help='output directory of test results')

parser.add_argument('--test_dataset', type=str, default='cylinder',
                    choices=['backstep', 'cylinder', 'jhtdb', 'dns_turb', 'sqg', 'tbl', 'twcf'],
                    help='test dataset to evaluate')
parser.add_argument('--plot_results', type=eval, default=True,
                    help="""Whether or not to plot predicted results.""")

parser.add_argument('--amp', type=eval, default=False, help='Wether to use auto mixed precision')
parser.add_argument('-a', '--arch', type=str, default='RAFT256', choices=['RAFT256'],
                    help='Type of architecture to use')
parser.add_argument('--batch_size', default=5, type=int)
parser.add_argument('--batch_size_test', default=1, type=int)
parser.add_argument('--split_size', default=1, type=int)
parser.add_argument('--offset', default=256, type=int,
                    help='interrogation window size')
parser.add_argument('--shift', default=64, type=int,
                    help='shift of interrogation window in px')

parser.add_argument('--iters', default=16, type=int,
                    help='number of update steps in ConvGRU')
parser.add_argument('--upsample', type=str, default='convex',
                    choices=['convex', 'bicubic', 'bicubic8', 'lanczos4', 'lanczos4_8'],
                    help="""Type of upsampling method""")
args = parser.parse_args([])

In [4]:
prefix = './PIG/'
# lamb-oseen flow
pathList = glob.glob(prefix+'oseen*.npz')
pathList.sort(key=lambda item: float(item[12:17]))
## sine flow
# pathList = glob.glob('sin*.npz',root_dir='./PIG')
# pathList.sort(key=lambda item: float(item[4:6]))
pathList
# [prefix + x for x in pathList]

['./PIG/oseen_100.0.npz',
 './PIG/oseen_250.0.npz',
 './PIG/oseen_500.0.npz',
 './PIG/oseen_1000.0.npz',
 './PIG/oseen_2000.0.npz',
 './PIG/oseen_3000.0.npz',
 './PIG/oseen_4000.0.npz',
 './PIG/oseen_5000.0.npz',
 './PIG/oseen_6000.0.npz',
 './PIG/oseen_7000.0.npz']

In [5]:
def computeEPE(gt:torch.Tensor, pred:torch.Tensor):
    """ gt: NxCxWxH or CxWxH
        pred: NxCxWxH or CxWxH
        return: float64 """
    if len(pred.shape) == 4:
        pred = pred.squeeze(0)
    if len(gt.shape) == 4:
        gt = gt.squeeze(0)

    epe = torch.sum(torch.square(pred-gt),dim=0).sqrt()
    return epe.mean().item()

In [15]:
# load model
model = RAFT256_SS(args=args).to(device)
checkpoint = torch.load('./results/RAFT256-PIV_SS_ProbClass2/ckpt.tar')
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

for path in pathList:
    # read flow field
    case = path
    # case = prefix+path
    print(case)
    # case = prefix+'oseen_1000.0.npz'
    data = np.load(case)
    element = ['img1', 'img2', 'u', 'v']
    img1, img2, u, v = [data[x] for x in element]
    flows = np.stack((u,v),axis=0)

    img1 = torch.from_numpy(img1).float()
    img1 = img1.repeat(1,1,1,1)
    img2 = torch.from_numpy(img2).float()
    img2 = img2.repeat(1,1,1,1)
    flows = torch.from_numpy(flows).float()
    flows = flows.unsqueeze(dim=0)

    # to gpu
    img1, img2, flows = [x.to(device) for x in [img1,img2,flows]]
    
    # reshape image
    img1 = img1 / 255
    img2 = img2 /255
    imgs = torch.stack((img1,img2),dim=1)
    imgs = torch.squeeze(imgs, dim=2)
    
    pred_flows = model(imgs, flows, args=args, evaluate=False)
    
    all_flow_iters = pred_flows[0]
    predicted_flows = all_flow_iters[-1]
    print(pred_flows[1][0], pred_flows[1][1], sep='\n')

    # vector field
    pred_flows = model(imgs, flows, args=args, evaluate=True)
    all_flow_iters = pred_flows[0]
    print(pred_flows[1][0], pred_flows[1][1], sep='\n')
    
    print('---------')

    # gt = np.stack((u,v),axis=0)
    # print(gt.shape)
    # print(all_flow_iters[-1].shape)
    # gt = torch.from_numpy(gt).to(device)
    # epe = computeEpe(gt,all_flow_iters[-1])
    # print(epe)
    # break


./PIG/oseen_100.0.npz


/home/wanxuan/miniconda3/envs/torch-cupy/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1678402374358/work/aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


tensor(0.0889, device='cuda:0', grad_fn=<AddBackward0>)
{'epe': 0.027963761240243912, '1px': 1.0, '3px': 1.0, '5px': 1.0}
tensor(0.0889, device='cuda:0', grad_fn=<AddBackward0>)
{'epe': 0.027966082096099854, '1px': 1.0, '3px': 1.0, '5px': 1.0}
---------
./PIG/oseen_250.0.npz
tensor(0.1260, device='cuda:0', grad_fn=<AddBackward0>)
{'epe': 0.040658604353666306, '1px': 1.0, '3px': 1.0, '5px': 1.0}
tensor(0.1255, device='cuda:0', grad_fn=<AddBackward0>)
{'epe': 0.040407922118902206, '1px': 1.0, '3px': 1.0, '5px': 1.0}
---------
./PIG/oseen_500.0.npz
tensor(0.1899, device='cuda:0', grad_fn=<AddBackward0>)
{'epe': 0.06032513454556465, '1px': 1.0, '3px': 1.0, '5px': 1.0}
tensor(0.1894, device='cuda:0', grad_fn=<AddBackward0>)
{'epe': 0.06011894345283508, '1px': 1.0, '3px': 1.0, '5px': 1.0}
---------
./PIG/oseen_1000.0.npz
tensor(0.3036, device='cuda:0', grad_fn=<AddBackward0>)
{'epe': 0.09502723813056946, '1px': 1.0, '3px': 1.0, '5px': 1.0}
tensor(0.3071, device='cuda:0', grad_fn=<AddBackward

In [18]:
model= RAFT256(args=args).to(device)
checkpoint = torch.load('./results/ckpt.tar')
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

for path in pathList:
    # read lamb-oseen flow field
    # case = prefix+path
    case = path
    print(case)
    # case = prefix+'oseen_1000.0.npz'
    data = np.load(case)
    element = ['img1', 'img2', 'u', 'v']
    img1, img2, u, v = [data[x] for x in element]
    flows = np.stack((u,v),axis=0)
    
    # Query only one sample from each sample set
    img1 = img1[0]
    img2 = img2[0]
    flows = flows[:,0,...]
    # print(img1.shape, img2.shape, flows.shape)

    img1 = torch.from_numpy(img1).float()
    img1 = img1.repeat(1,1,1,1)
    img2 = torch.from_numpy(img2).float()
    img2 = img2.repeat(1,1,1,1)
    flows = torch.from_numpy(flows).float()
    flows = flows.unsqueeze(dim=0)

    # to gpu
    img1, img2, flows = [x.to(device) for x in [img1,img2,flows]]
    
    # reshape image
    img1 = img1 / 255
    img2 = img2 /255
    imgs = torch.stack((img1,img2),dim=1)
    imgs = torch.squeeze(imgs, dim=2)

    pred_flows = model(imgs, flows, args=args)
    all_flow_iters = pred_flows[0]
    predicted_flows = all_flow_iters[-1]
    print(pred_flows[1][0], pred_flows[1][1], sep='\n')
    
    # SHOW_RESULTS = True
    # if SHOW_RESULTS:
    #     predFlow = predicted_flows.detach().cpu().numpy()
    #     print(predFlow.shape)
    #     plt.quiver(predFlow[0,0,...], predFlow[0,1,...])
    #     plt.figure(figsize=(10,10))
    #     plt.show()

    print('------')

./PIG/oseen_100.0.npz
tensor(0.2069, device='cuda:0', grad_fn=<AddBackward0>)
{'epe': 0.06655582785606384, '1px': 1.0, '3px': 1.0, '5px': 1.0}
------
./PIG/oseen_250.0.npz
tensor(0.2348, device='cuda:0', grad_fn=<AddBackward0>)
{'epe': 0.07670526206493378, '1px': 1.0, '3px': 1.0, '5px': 1.0}
------
./PIG/oseen_500.0.npz
tensor(0.3435, device='cuda:0', grad_fn=<AddBackward0>)
{'epe': 0.11152881383895874, '1px': 1.0, '3px': 1.0, '5px': 1.0}
------
./PIG/oseen_1000.0.npz
tensor(0.5021, device='cuda:0', grad_fn=<AddBackward0>)
{'epe': 0.1623324751853943, '1px': 1.0, '3px': 1.0, '5px': 1.0}
------
./PIG/oseen_2000.0.npz
tensor(0.6692, device='cuda:0', grad_fn=<AddBackward0>)
{'epe': 0.218244731426239, '1px': 0.99871826171875, '3px': 1.0, '5px': 1.0}
------
./PIG/oseen_3000.0.npz
tensor(0.9687, device='cuda:0', grad_fn=<AddBackward0>)
{'epe': 0.3182051181793213, '1px': 0.9839935302734375, '3px': 1.0, '5px': 1.0}
------
./PIG/oseen_4000.0.npz
tensor(1.3471, device='cuda:0', grad_fn=<AddBackwa

In [18]:
from unliteflownet.model.models import Network, estimate

model = Network().cuda(0)
ckpt = torch.load('./results/UnLiteFlowNet-PIV_Problem2/ckpt.tar')
model.load_state_dict(ckpt['model_state_dict'])
model.eval()

for path in pathList:
    # read lamb-oseen flow field
    # case = prefix+path
    case = path
    print(case)
    # case = prefix+'oseen_1000.0.npz'
    data = np.load(case)
    element = ['img1', 'img2', 'u', 'v']
    img1, img2, u, v = [data[x] for x in element]
    flows = np.stack((u,v),axis=0)

    img1 = torch.from_numpy(img1).float()
    img1 = img1.repeat(1,1,1,1)
    img2 = torch.from_numpy(img2).float()
    img2 = img2.repeat(1,1,1,1)
    flows = torch.from_numpy(flows).float()
    flows = flows.unsqueeze(dim=0)

    # to gpu
    img1, img2, flows = [x.to(device) for x in [img1,img2,flows]]
    
    # reshape image
    img1 = img1 / 255
    img2 = img2 /255
    imgs = torch.stack((img1,img2),dim=1)
    imgs = torch.squeeze(imgs, dim=2)

    # pred_flows = model(imgs, flows, args=args)
    # all_flow_iters = pred_flows[0]
    # predicted_flows = all_flow_iters[-1]
    # print(pred_flows[1][0], pred_flows[1][1], sep='\n')

    output_forward = estimate(img1, img2, model, train=True)
    # loss = criterion_val(output_forward[-1],flows)
    # validation_EPE += loss.item()*image1.shape[0]
    epe = computeEPE(output_forward[-1],flows)
    print(f"epe: {epe}")

    print('------')


./PIG/oseen_100.0.npz
epe: 0.06867652386426926
------
./PIG/oseen_250.0.npz
epe: 0.08900302648544312
------
./PIG/oseen_500.0.npz
epe: 0.09371756762266159
------
./PIG/oseen_1000.0.npz
epe: 0.11205799132585526
------
./PIG/oseen_2000.0.npz
epe: 0.1699695587158203
------
./PIG/oseen_3000.0.npz
epe: 0.2768195867538452
------
./PIG/oseen_4000.0.npz
epe: 0.4395485520362854
------
./PIG/oseen_5000.0.npz
epe: 0.6480282545089722
------
./PIG/oseen_6000.0.npz
epe: 1.142491340637207
------
./PIG/oseen_7000.0.npz
epe: 1.6230933666229248
------
